In [2]:
# !pip install gym
# !pip install pygame
import gym
import time
from random import randint

     |████████████████████████████████| 696kB 4.2MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7MB 39.6MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/52/58/66566c5bd7688f2cd792017bfcae26010c523786721a4ccb9c
Successfully built gym
  Found existing installation: cloudpickle 0.8.1
    Uninstalling cloudpickle-0.8.1:
      Successfully uninstalled cloudpickle-0.8.1
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
     |████████████████████████████████| 21.2MB 4.0MB/s eta 0:00:01


# Cargo el entorno

In [5]:
env = gym.make('CartPole-v1')

# Simulación

In [7]:
for _ in range(10):
    obs = env.reset()
    done = False
    while not done:
        policy = randint(0,1)
        obs, reward, done, info = env.step(policy)
        # print(obs)
        env.render()
        time.sleep(0.05)
        if done:
            env.reset()
env.close()

error: No available video device

In [67]:
import numpy as np
def createEpsilonGreedyPolicy(Q, epsilon, num_actions):
    """
    Creates an epsilon-greedy policy based
    on a given Q-function and epsilon.
       
    Returns a function that takes the state
    as an input and returns the probabilities
    for each action in the form of a numpy array 
    of length of the action space(set of possible actions).
    """
    def policyFunction(state):
   
        Action_probabilities = np.ones(num_actions,
                dtype = float) * epsilon / num_actions
    
        state_tup = state[0] , state[1] , state[2] , state[3] 
        
        best_action = np.argmax(Q[tup])
        Action_probabilities[best_action] += (1.0 - epsilon)
        return Action_probabilities
   
    return policyFunction

In [72]:
from collections import defaultdict
import gym
import itertools
import matplotlib
import matplotlib.style
import numpy as np
import pandas as pd
import sys

def qLearning(env, num_episodes, discount_factor = 1.0,
                            alpha = 0.6, epsilon = 0.1):
    """
    Q-Learning algorithm: Off-policy TD control.
    Finds the optimal greedy policy while improving
    following an epsilon-greedy policy"""
       
    # Action value function
    # A nested dictionary that maps
    # state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
       
    # Create an epsilon greedy policy function
    # appropriately for environment action space
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n)
       
    # For every episode
    for ith_episode in range(num_episodes):
           
        # Reset the environment and pick the first action
        state = env.reset()
           
        for t in itertools.count():
               
            # get probabilities of all actions from current state
            action_probabilities = policy(state)
   
            # choose action according to 
            # the probability distribution
            action = np.random.choice(np.arange(
                      len(action_probabilities)),
                       p = action_probabilities)
            
   
            # take action and get reward, transit to next state
            next_state, reward, done, _ = env.step(action)
           
            state_tup = state[0] , state[1] , state[2] , state[3] 
            sext_state_tup = next_state[0] , next_state[1] , next_state[2] , next_state[3] 
               
            # TD Update
            best_next_action = np.argmax(Q[sext_state_tup])    
            td_target = reward + discount_factor * Q[sext_state_tup][best_next_action]
            td_delta = td_target - Q[state_tup][action]
            Q[state_tup][action] += alpha * td_delta
   
            # done is True if episode terminated   
            if done:
                break
                   
            state = next_state
       
    return Q

In [73]:
Q = qLearning(env, 1000)